# Simple Scenario for interaction by chat

This note books demonstrates how a Scenario is created through EMISSOR using the driver_util functions.
A simple chat loop gets use input and gives dummy output. Each utterance is represented as a new TextSignal.
The textSignals are added to the scenario. At the end of the chat, the scenario is saved to disk.

## Importing EMISSOR

In [3]:
import emissor as em
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

### Others

import uuid
import time
from datetime import datetime

## Importing the driver utility functions for turning a chat into a scenario in EMSISSOR

In [4]:
import sys
import os
src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)
print(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util

/Users/piek/PycharmProjects/cltl-chatbots/src


## Standard initialisation of a scenario

The following standard steps are necessary to initialise a scenarion in EMISSOR, where we set the location and time as a context.
We also hardcode a name for the system and the user. In a more advanced system the user name and identity needs to be determined by the system and through interaction.

In [5]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")


### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
# Find the repository root dir
parent, dir_name = (d_util.__file__, "_")
while dir_name and dir_name != "src":
    parent, dir_name = os.path.split(parent)
root_dir = parent
scenario_path = os.path.abspath(os.path.join(root_dir, 'data'))

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)
    
### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario_ctrl = scenarioStorage.create_scenario(scenario_id, int(time.time() * 1e3), None, AGENT)

Directories for 2021-11-19-09:29:42 created in /Users/piek/PycharmProjects/cltl-chatbots/data


You can now check the data folder to see that a subfolder is created based on the system date and time for the EMISSOR data structure. It contains subsubfolders for storing media data. All meta data on media signals is stored in json file per media. Here you now only see the json file for the complete scenario. 

The json file has a temporal rule to align media signals and it has the basic context information for the scenario.

## Have a chat conversation

Next we can have a simple chat with a system that only gives dummy responses. Each utterance is saved as a TextSignal and added the to scenaario.

In [6]:
### First prompt
response = "How are you doing "+HUMAN_NAME
textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, response, AGENT)
scenario_ctrl.append_signal(textSignal)

print(AGENT + ": " + response)

utterance = input("\n")
print(HUMAN_NAME + ": " + utterance)

while not (utterance.lower() == "stop" or utterance.lower() == "bye"):
    textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, utterance, HUMAN_ID)
    scenario_ctrl.append_signal(textSignal)

    # Create the response from the system and store this as a new signal
    # We could use the throughts to respond
    # @TODO generate a response from the thoughts

    utterance = "Say whatever you want to say " + HUMAN_NAME + "\nSay stop or bye to quit"
    response = utterance[::-1]
    print(AGENT + ": " + utterance)
    textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, utterance, AGENT)
    scenario_ctrl.append_signal(textSignal)

    # Getting the next input signals
    utterance = input("\n")


Leolani2: How are you doing Stranger



 I fjdfhgjkdfngamndfb


Stranger: I fjdfhgjkdfngamndfb
Leolani2: Say whatever you want to say Stranger
Say stop or bye to quit



 hdfjkhfkjghdkgjhfd


Leolani2: Say whatever you want to say Stranger
Say stop or bye to quit



 jghfdkjgakdf


Leolani2: Say whatever you want to say Stranger
Say stop or bye to quit



 stop


## Save the scenario to the data folder

When we stop the conversation the scenario is saved in memory. We need to save it, which updates the scenario json but also dumps the medata for any media Signal in as correspond json file. In this case, only a text.json is created with the utterances.

In [7]:
scenario_ctrl.scenario.ruler.end = int(time.time() * 1e3)
scenarioStorage.save_scenario(scenario_ctrl)

## End of the notebook